In [ ]:
#|default_exp core

# Claudio source code

This is the 'literate' source code for Claudio. You can view the fully rendered version of the source [here](https://answerdotai.github.io/claudio/core.html). It is converted into a Python module using [nbdev](https://nbdev.fast.ai/). The goal of this source code is to both create the Python module, and also to teach the reader *how* it is created, without assuming much existing knowledge about Claude's API.

## Setup

In [ ]:
import os
# os.environ['ANTHROPIC_LOG'] = 'debug'

To print every HTTP request and response in full, uncomment the above line. This functionality is provided by Anthropic's SDK.

In [ ]:
#| export
import tokenize, ast, inspect, inspect, typing
import xml.etree.ElementTree as ET, json
from collections import abc

from anthropic import Anthropic
from anthropic.types import Usage, TextBlock, Message
from anthropic.types.beta.tools import ToolsBetaMessage, tool_use_block
from inspect import Parameter
from io import BytesIO
try: from IPython.display import Markdown,HTML
except: Markdown,HTML=None,None

from fastcore.docments import docments
from fastcore.utils import *

In [ ]:
#| exports
models = 'claude-3-opus-20240229','claude-3-sonnet-20240229','claude-3-haiku-20240307'

These are the current versions of Anthropic's model at the time of writing.

In [ ]:
#| export
empty = Parameter.empty

In [ ]:
model = models[-1]

For examples, we'll use Haiku, since it's fast and cheap (and surprisingly good!)

## Client

In [ ]:
cli = Anthropic()

This is what Anthropic's SDK provides for interacting with Python. To use it, pass it a list of *messages*, with *content* and a *role*. The roles should alternate between *user* and *assistant*.

In [ ]:
m = {'role': 'user', 'content': "I'm Jeremy"}
r = cli.messages.create(messages=[m], model=model, max_tokens=100)
r

Message(id='msg_0166BSyHCJzC2qRoaqEgcaLa', content=[TextBlock(text="It's nice to meet you Jeremy! I'm Claude, an AI assistant created by Anthropic. How can I help you today?", type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=10, output_tokens=31))

In [ ]:
#| exports
def mk_msg(content, role='user', **kw):
    "Helper to create a `dict` appropriate for a Claude message"
    if hasattr(content, 'content'): content,role = content.content,content.role
    if isinstance(content, abc.Mapping): content=content['content']
    return dict(role=role, content=content, **kw)

We make things a bit more convenient by writing a function to create these messages for us.

As you see from the source, `mk_msg` doesn't only handle `str` for `content`, but can also deal with a `dict` or an object containing `content` (such as for from assistant responses).

In [ ]:
prompt = "I'm Jeremy"
m = mk_msg(prompt)
m

{'role': 'user', 'content': "I'm Jeremy"}

In [ ]:
r = cli.messages.create(messages=[m], model=model, max_tokens=100)
r

Message(id='msg_01XfjfhTtJZVf4uZXvVdGMGr', content=[TextBlock(text="It's nice to meet you, Jeremy! How can I assist you today?", type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=10, output_tokens=19))

In [ ]:
#| exports
def find_block(r, blk_type=TextBlock):
    "Find the first block of type `blk_type` in `r.content`"
    return first(o for o in r.content if isinstance(o,blk_type))

This makes it easier to grab the needed parts of Claude's responses, which can include multiple pieces of content. By default, we look for the first text block.

In [ ]:
find_block(r)

TextBlock(text="It's nice to meet you, Jeremy! How can I assist you today?", type='text')

In [ ]:
#| exports
def contents(r):
    "Helper to get the contents from Claude response `r`"
    return find_block(r).text.strip()

For display purposes, we often just want to show the text itself.

In [ ]:
contents(r)

"It's nice to meet you, Jeremy! How can I assist you today?"

In [ ]:
#| exports
def mk_msgs(msgs, **kw):
    "Helper to set 'assistant' role on alternate messages"
    if isinstance(msgs,str): msgs=[msgs]
    return [mk_msg(o, ('user','assistant')[i%2], **kw) for i,o in enumerate(msgs)]

LLMs, including Claude, don't actually have state, but instead dialogs are created by passing back all previous prompts and responses every time. With Claude, they always alternate *user* and *assistant*. Therefore we create a function to make it easier to build up these dialog lists.

In [ ]:
msgs = mk_msgs([prompt, r, 'I forgot my name. Can you remind me please?'])
msgs

[{'role': 'user', 'content': "I'm Jeremy"},
 {'role': 'assistant',
  'content': [TextBlock(text="It's nice to meet you, Jeremy! How can I assist you today?", type='text')]},
 {'role': 'user', 'content': 'I forgot my name. Can you remind me please?'}]

Now, if we pass this list of messages to Claude, the model treats it as a conversation to respond to.

In [ ]:
r = cli.messages.create(messages=msgs, model=model, max_tokens=200)
r

Message(id='msg_01PZdDuPReddF627z6fSdUaU', content=[TextBlock(text="I'm afraid I don't actually know your name. I only know that you introduced yourself as Jeremy. If you've forgotten your own name, I don't have a way to remind you of it. Perhaps you could check some form of identification or think back to when you last used your name.", type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=43, output_tokens=63))

In [ ]:
@patch
def _repr_html_(self:(ToolsBetaMessage,Message)):
    det = '</li>\n<li>'.join(f'{k}: {v}' for k,v in self.dict().items())
    return f"""{contents(self)}
<details><ul><li>
{det}
</ul></li></details>"""

Jupyter looks for a `_repr_html_` method in displayed objects; we add this in order to display just the content text, and collapse full details into a hideable section. Note that `patch` is from `fastcore`, and is used to add (or replace) functionality in an existing class.

In [ ]:
r

Message(id='msg_01FidCYYvZsh8scXhxK7JiYR', content=[TextBlock(text="I'm afraid I don't actually know your name - you introduced yourself to me as Jeremy, but if you've forgotten your own name, I don't have a way to remind you of it. As an AI, I don't have personal information about you stored. I can only respond based on what you tell me directly in our conversation.", type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=78, output_tokens=72))

One key part of the response is the `usage` key, which tells us how many tokens we used:

In [ ]:
r.usage

Usage(input_tokens=78, output_tokens=72)

In [ ]:
#| exports
def usage(inp=0, out=0):
    "Slightly more concise version of `Usage`"
    return Usage(input_tokens=inp, output_tokens=out)

The constructor provided by Anthropic is rather verbose, so we clean it up a bit, using a lowercase version of the name.

In [ ]:
usage(5)

Usage(input_tokens=5, output_tokens=0)

In [ ]:
#| exports
@patch(as_prop=True)
def total(self:Usage): return self.input_tokens+self.output_tokens

Adding a `total` property to `Usage` makes it easier to see how many tokens we've used up altogether.

In [ ]:
usage(5,1).total

6

In [ ]:
#| exports
@patch
def __repr__(self:Usage): return f'In: {self.input_tokens}; Out: {self.output_tokens}; Total: {self.total}'

In [ ]:
r.usage

In: 78; Out: 72; Total: 150

In [ ]:
#| exports
@patch
def __add__(self:Usage, b):
    return usage(self.input_tokens+b.input_tokens, self.output_tokens+b.output_tokens)

In [ ]:
r.usage+r.usage

In: 156; Out: 144; Total: 300

In [ ]:
#| exports
class Client:
    def __init__(self, model, cli=None):
        "Basic Anthropic messages client"
        self.model,self.use = model,Usage(input_tokens=0,output_tokens=0)
        self.c = (cli or Anthropic())

In [ ]:
c = Client(model)
c.use

In: 0; Out: 0; Total: 0

In [ ]:
#| exports
@patch
def _r(self:Client, r:ToolsBetaMessage):
    "Store the result of the message and accrue total usage"
    self.result = r
    self.use += r.usage
    return r

In [ ]:
c._r(r)
c.use

In: 78; Out: 72; Total: 150

In [ ]:
#| exports
@patch
def __call__(self:Client, msgs, sp='', temp=0, maxtok=4096, stop=None, **kw):
    "Make a call to Claude without streaming"
    r = self.c.beta.tools.messages.create(
        model=self.model, messages=mk_msgs(msgs), max_tokens=maxtok, system=sp, temperature=temp, stop_sequences=stop, **kw)
    return self._r(r)

In [ ]:
c('Hi')

ToolsBetaMessage(id='msg_015UMRiKTrQxrrUpg2D13QuL', content=[TextBlock(text='Hello! How can I assist you today?', type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=In: 8; Out: 12; Total: 20)

In [ ]:
c.use

In: 16; Out: 24; Total: 40

In [ ]:
#| exports
@patch
def stream(self:Client, msgs, sp='', temp=0, maxtok=4096, stop=None, **kw):
    "Make a call to Claude, streaming the result"
    with self.c.messages.stream(model=self.model, messages=mk_msgs(msgs), max_tokens=maxtok,
                                system=sp, temperature=temp, stop_sequences=stop, **kw) as s:
        yield from s.text_stream
        return self._r(s.get_final_message())

In [ ]:
for o in c.stream('Hi'): print(o, end='')

Hello! How can I assist you today?

In [ ]:
c.use

In: 24; Out: 36; Total: 60

## Tool use

In [ ]:
#| exports
def _types(t:type)->tuple[str,Optional[str]]:
    "Tuple of json schema type name and (if appropriate) array item name"
    tmap = {int:"integer", float:"number", str:"string", bool:"boolean", list:"array", dict:"object"}
    if getattr(t, '__origin__', None) in  (list,tuple): return "array", tmap.get(t.__args__[0], "object")
    else: return tmap.get(t, "object"), None

In [ ]:
_types(list[int]), _types(int)

(('array', 'integer'), ('integer', None))

In [ ]:
def silly_sum(
    # First thing to sum
    a:int,
    # Second thing to sum
    b:int=1,
    # A pointless argument
    c:list[int]=None,
# The sum of the inputs
) -> int:
    "Adds a + b"
    return a + b

In [ ]:
d = docments(silly_sum, full=True)
d

```json
{ 'a': { 'anno': <class 'int'>,
         'default': <class 'inspect._empty'>,
         'docment': 'First thing to sum'},
  'b': {'anno': <class 'int'>, 'default': 1, 'docment': 'Second thing to sum'},
  'c': {'anno': list[int], 'default': None, 'docment': 'A pointless argument'},
  'return': { 'anno': <class 'int'>,
              'default': <class 'inspect._empty'>,
              'docment': 'The sum of the inputs'}}
```

In [ ]:
#| exports
def _param(name, info):
    "json schema parameter given `name` and `info` from docments full dict"
    paramt,itemt = _types(info.anno)
    pschema = dict(type=paramt, description=info.docment)
    if itemt: pschema["items"] = {"type": itemt}
    if info.default is not empty: pschema["default"] = info.default
    return pschema

In [ ]:
n,o = first(d.items())
print(n,'//', o)
_param(n, o)

a // {'docment': 'First thing to sum', 'anno': <class 'int'>, 'default': <class 'inspect._empty'>}


{'type': 'integer', 'description': 'First thing to sum'}

In [ ]:
#| exports
def get_schema(f):
    d = docments(f, full=True)
    ret = d.pop('return')
    paramd = {
        'type': "object",
        'properties': {n:_param(n,o) for n,o in d.items()},
        'required': [n for n,o in d.items() if o.default is empty]
    }
    desc = f.__doc__
    if ret.anno is not empty: desc += f'\n\nReturns:\n- type: {_types(ret.anno)[0]}'
    if ret.docment: desc += f'\n- description: {ret.docment}'
    return dict(name=f.__name__, description=desc, input_schema=paramd)

In [ ]:
s = get_schema(silly_sum)
desc = s.pop('description')
print(desc)
s

Adds a + b

Returns:
- type: integer
- description: The sum of the inputs


{'name': 'silly_sum',
 'input_schema': {'type': 'object',
  'properties': {'a': {'type': 'integer', 'description': 'First thing to sum'},
   'b': {'type': 'integer',
    'description': 'Second thing to sum',
    'default': 1},
   'c': {'type': 'array',
    'description': 'A pointless argument',
    'items': {'type': 'integer'},
    'default': None}},
  'required': ['a']}}

In [ ]:
def sums(
    # First thing to sum
    a:int,
    # Second thing to sum
    b:int=1
# The sum of the inputs
) -> int:
    "Adds a + b"
    return a + b

In [ ]:
pr = "What is 6+3?"
sp = "You must use the `sums` function instead of adding yourself, but don't mention what tools you use."
tools=[get_schema(sums)]

In [ ]:
msgs = mk_msgs(pr)
r = c(msgs, sp=sp, tools=tools)

In [ ]:
#| exports
def mk_ns(*funcs:list[callable]) -> dict[str,callable]:
    "Create a `dict` of name to function in `funcs`, to use as a namespace"
    return {f.__name__:f for f in funcs}

In [ ]:
#| exports
def find_tool(r) -> tool_use_block.ToolUseBlock:
    return first(o for o in r.content if isinstance(o,tool_use_block.ToolUseBlock))

In [ ]:
#| exports
def call_func(tr, ns=None):
    "Call the function in the tool response `tr`, using namespace `ns`"
    if ns is None: ns=globals()
    if not isinstance(ns, abc.Mapping): ns = mk_ns(*ns)
    fc = find_tool(r)
    return ns[fc.name](**fc.input)

In [ ]:
# list of functions you can call
funcs = [sums]

In [ ]:
res = call_func(r, ns=funcs)
res

9

In [ ]:
msgs.append(mk_msg(r.content, role=r.role))

In [ ]:
#| exports
def mk_toolres(r, res=None, ns=None):
    "Create a `tool_result` message from response `r`"
    if not hasattr(r, 'content'): return r
    tool = first(o for o in r.content if isinstance(o,tool_use_block.ToolUseBlock))
    if not tool: return r
    if res is None: res = call_func(r, ns)
    tr = dict(type="tool_result", tool_use_id=tool.id, content=str(res))
    return mk_msg([tr])

In [ ]:
tr = mk_toolres(r, res=res, ns=[sums])
tr

{'role': 'user',
 'content': [{'type': 'tool_result',
   'tool_use_id': 'toolu_019CrmHStEpRjQ4JcZprwxBr',
   'content': '9'}]}

In [ ]:
msgs.append(tr)

In [ ]:
contents(c(msgs, sp=sp, tools=tools))

'The sum of 6 and 3 is 9.'

## Chat

In [ ]:
#| exports
class Chat:
    def __init__(self, model=None, cli=None):
        "Anthropic chat client"
        assert model or cli
        self.c = (cli or Client(model))
        self.h = []
    
    def __call__(self, pr, sp='', temp=0, maxtok=4096, stop=None, ns=None, tools=None, **kw):
        if ns is None: ns=tools
        self.h = mk_msgs(self.h + [mk_toolres(pr, ns=ns)])
        if tools: kw['tools'] = [get_schema(o) for o in tools]
        res = self.c(self.h, sp=sp, temp=temp, maxtok=maxtok, stop=stop, **kw)
        self.h.append(mk_msg(res, role='assistant'))
        return res

In [ ]:
c = Chat(model)

In [ ]:
c("I'm Jeremy")
contents(c("What's my name?"))

'Your name is Jeremy, as you told me earlier.'

In [ ]:
pr = "What is 6+3?"
sp = "You must use the `sums` function instead of adding yourself, but don't mention what tools you use."
tools=[sums]

In [ ]:
r = c(pr, sp=sp, tools=tools)
r

ToolsBetaMessage(id='msg_011d36YYZNFGyzd7C9EmBYXT', content=[TextBlock(text="Okay, let's use the provided tool to calculate 6 + 3:", type='text'), ToolUseBlock(id='toolu_011gDmiieecyMbDGc56omWUA', input={'a': 6, 'b': 3}, name='sums', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=In: 471; Out: 88; Total: 559)

In [ ]:
c(r, sp=sp, tools=tools)

ToolsBetaMessage(id='msg_019oC32MFxLrPV1FEsg9MmpA', content=[TextBlock(text='The sum of 6 and 3 is 9.', type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=In: 571; Out: 18; Total: 589)

## XML helpers

In [ ]:
#| exports
def hl_md(s, lang='xml'):
    "Syntax highlight `s` using `lang`"
    if Markdown: return Markdown(f'```{lang}\n{s}\n```')
    print(s)

In [ ]:
#| exports
def to_xml(node, hl=False):
    "Convert `node` to an XML string"
    def mk_el(tag, cs, attrs):
        el = ET.Element(tag, attrib=attrs)
        if isinstance(cs, list): el.extend([mk_el(*o) for o in cs])
        elif cs is not None: el.text = str(cs)
        return el

    root = mk_el(*node)
    ET.indent(root)
    res = ET.tostring(root, encoding='unicode')
    return hl_md(res) if hl else res

In [ ]:
#| exports
def xt(tag, c=None, **kw):
    "Helper to create appropriate data structure for `to_xml`"
    kw = {k.lstrip('_'):str(v) for k,v in kw.items()}
    return tag,c,kw

In [ ]:
#| exports
g = globals()
tags = 'div','img','h1','h2','h3','h4','h5','p','hr','span','html'
for o in tags: g[o] = partial(xt, o)

In [ ]:
a = html([
    p('This is a paragraph'),
    hr(),
    xt('x-custom', foo='bar'),
    img(src='http://example.prg'),
    div([
        h1('This is a header'),
        h2('This is a sub-header', style='k:v'),
    ], _class='foo')
])

In [ ]:
to_xml(a, True)

```xml
<html>
  <p>This is a paragraph</p>
  <hr />
  <x-custom foo="bar" />
  <img src="http://example.prg" />
  <div class="foo">
    <h1>This is a header</h1>
    <h2 style="k:v">This is a sub-header</h2>
  </div>
</html>
```

In [ ]:
#|export
def json_to_xml(d:dict, rnm:str)->str:
    "Convert `d` to XML with root name `rnm`"
    root = ET.Element(rnm)
    def build_xml(data, parent):
        if isinstance(data, dict):
            for key, value in data.items(): build_xml(value, ET.SubElement(parent, key))
        elif isinstance(data, list):
            for item in data: build_xml(item, ET.SubElement(parent, 'item'))
        else: parent.text = str(data)
    build_xml(d, root)
    ET.indent(root)
    return ET.tostring(root, encoding='unicode')

In [ ]:
a = dict(surname='Howard', firstnames=['Jeremy','Peter'],
         address=dict(state='Queensland',country='Australia'))
print(json_to_xml(a, 'person'))

<person>
  <surname>Howard</surname>
  <firstnames>
    <item>Jeremy</item>
    <item>Peter</item>
  </firstnames>
  <address>
    <state>Queensland</state>
    <country>Australia</country>
  </address>
</person>


## Images

Not done yet.

## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()